In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.functional as F
import pickle

In [2]:
# file = open('googleNet_embeddings.pkl', 'rb')
# embeddings = torch.load('../matrix_factorization/InceptionNet_embeddings.pkl')
embeddings = np.load("../matrix_factorization/embeddings_alt_arch_with_decoder.npy")
embeddings

array([[-0.0866257 ,  0.00861705, -0.02768877, ...,  0.00064043,
         0.03820464, -0.0443062 ],
       [-0.00733163,  0.00637373, -0.00079712, ...,  0.00984534,
        -0.01724906,  0.04450938],
       [-0.01434905,  0.0084365 , -0.01561192, ...,  0.00348288,
         0.00016413,  0.02360854],
       ...,
       [-0.01827695,  0.00692758, -0.01209303, ...,  0.00339431,
         0.00328189,  0.02521952],
       [ 0.00211651,  0.00280362, -0.0056607 , ...,  0.00286982,
        -0.01519183,  0.05249388],
       [ 0.02291233,  0.00477167,  0.00636004, ...,  0.00610208,
        -0.03793249,  0.08019338]])

In [3]:
train_ids = torch.load("../dataset/train_book_id.pkl")
test_ids = torch.load("../dataset/test_book_id.pkl")
print(train_ids.size())
print(test_ids.size())


torch.Size([4000])
torch.Size([1000])


In [4]:
df = pd.read_csv("../dataset/books_with_genres.csv")
df['is_fiction'] = df['binary_category'].apply(lambda x: 1 if x=='fiction' else 0)
df.head()

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,...,"mystery, thriller, crime",poetry,romance,non-fiction,children,young-adult,"comics, graphic",category,binary_category,is_fiction
0,NaN,7,['189911'],US,eng,"[{'count': '58', 'name': 'to-read'}, {'count':...",B00071IKUY,False,4.03,NaN,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,"fantasy, paranormal",fiction,1
1,0842379428,566,[],US,eng,"[{'count': '6393', 'name': 'to-read'}, {'count...",NaN,False,4.26,B000FCKCJC,...,NaN,NaN,NaN,163.0,NaN,NaN,NaN,non-fiction,non-fiction,0
2,0590417010,193,[],US,eng,"[{'count': '450', 'name': 'to-read'}, {'count'...",NaN,False,4.43,B017RORXNI,...,NaN,NaN,NaN,2.0,109.0,1.0,NaN,children,non-fiction,0
3,1400041694,44,[],US,en-US,"[{'count': '362', 'name': 'to-read'}, {'count'...",NaN,False,3.75,B002OTKEP6,...,NaN,NaN,NaN,37.0,NaN,NaN,NaN,non-fiction,non-fiction,0
4,NaN,1,[],US,eng,"[{'count': '8', 'name': 'to-read'}, {'count': ...",NaN,False,3.83,NaN,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,poetry,non-fiction,0


In [5]:
X_train = embeddings[0:4000]
X_test = embeddings[4000:]
print(X_train.shape)
print(X_test.shape)

(4000, 50)
(1000, 50)


In [6]:
y_train = df["is_fiction"][0:4000]
y_test = df["is_fiction"][4000:]
print(y_train.shape)
print(y_test.shape)

(4000,)
(1000,)


In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score,accuracy_score, precision_score, recall_score
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
score = f1_score(y_test, preds)
accuracy = accuracy_score(y_test, preds)
precision = precision_score(y_test, preds)
recall = recall_score(y_test, preds)
print('f1_score: ', score)
print('accuarcy: ', accuracy)
print('precision: ', precision)
print('recall: ', recall)

f1_score:  0.7653758542141231
accuarcy:  0.691
precision:  0.7411764705882353
recall:  0.7912087912087912


In [8]:
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
clf = BaggingClassifier(base_estimator=SVC(), n_estimators=10, random_state=0).fit(X_train, y_train)
preds = clf.predict(X_test)
score = f1_score(y_test, preds)
accuracy = accuracy_score(y_test, preds)
precision = precision_score(y_test, preds)
recall = recall_score(y_test, preds)
print('f1_score: ', score)
print('accuarcy: ', accuracy)
print('precision: ', precision)
print('recall: ', recall)

f1_score:  0.7782546494992848
accuarcy:  0.69
precision:  0.7148488830486203
recall:  0.8540031397174255


In [9]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train, y_train)
preds = clf.predict(X_test)
score = f1_score(y_test, preds)
accuracy = accuracy_score(y_test, preds)
precision = precision_score(y_test, preds)
recall = recall_score(y_test, preds)
print('f1_score: ', score)
print('accuarcy: ', accuracy)
print('precision: ', precision)
print('recall: ', recall)

f1_score:  0.775330396475771
accuarcy:  0.694
precision:  0.7282758620689656
recall:  0.8288854003139717
